# Downloading the pretrained models.

In [1]:
import os
import sys
import numpy as np
from PIL import Image

sys.path.append("../")
pretrained_model_dir = "./pretrained_models"

if not os.path.exists(pretrained_model_dir):
    os.mkdir(pretrained_model_dir)

In [ ]:
! cd pretrained_models && wget "http://vllab1.ucmerced.edu/~yli62/CartoonGAN/pytorch_pth/Hayao_net_G_float.pth"

In [ ]:
! cd pretrained_models && wget "http://vllab1.ucmerced.edu/~yli62/CartoonGAN/pytorch_pth/Hosoda_net_G_float.pth"

In [ ]:
! cd pretrained_models && wget "http://vllab1.ucmerced.edu/~yli62/CartoonGAN/pytorch_pth/Paprika_net_G_float.pth"

In [ ]:
! cd pretrained_models && wget "http://vllab1.ucmerced.edu/~yli62/CartoonGAN/pytorch_pth/Shinkai_net_G_float.pth"

## Our transformer model

In [3]:
import argparse
import torchvision.utils as vutils

In [2]:
# networks/Transformer.py
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torchvision.transforms as transforms

class Transformer(nn.Module):
    def __init__(self):
        super(Transformer, self).__init__()
        #
        self.refpad01_1 = nn.ReflectionPad2d(3)
        self.conv01_1 = nn.Conv2d(3, 64, 7)
        self.in01_1 = InstanceNormalization(64)
        # relu
        self.conv02_1 = nn.Conv2d(64, 128, 3, 2, 1)
        self.conv02_2 = nn.Conv2d(128, 128, 3, 1, 1)
        self.in02_1 = InstanceNormalization(128)
        # relu
        self.conv03_1 = nn.Conv2d(128, 256, 3, 2, 1)
        self.conv03_2 = nn.Conv2d(256, 256, 3, 1, 1)
        self.in03_1 = InstanceNormalization(256)
        # relu

        # res block 1
        self.refpad04_1 = nn.ReflectionPad2d(1)
        self.conv04_1 = nn.Conv2d(256, 256, 3)
        self.in04_1 = InstanceNormalization(256)
        # relu
        self.refpad04_2 = nn.ReflectionPad2d(1)
        self.conv04_2 = nn.Conv2d(256, 256, 3)
        self.in04_2 = InstanceNormalization(256)
        # + input

        # res block 2
        self.refpad05_1 = nn.ReflectionPad2d(1)
        self.conv05_1 = nn.Conv2d(256, 256, 3)
        self.in05_1 = InstanceNormalization(256)
        # relu
        self.refpad05_2 = nn.ReflectionPad2d(1)
        self.conv05_2 = nn.Conv2d(256, 256, 3)
        self.in05_2 = InstanceNormalization(256)
        # + input

        # res block 3
        self.refpad06_1 = nn.ReflectionPad2d(1)
        self.conv06_1 = nn.Conv2d(256, 256, 3)
        self.in06_1 = InstanceNormalization(256)
        # relu
        self.refpad06_2 = nn.ReflectionPad2d(1)
        self.conv06_2 = nn.Conv2d(256, 256, 3)
        self.in06_2 = InstanceNormalization(256)
        # + input

        # res block 4
        self.refpad07_1 = nn.ReflectionPad2d(1)
        self.conv07_1 = nn.Conv2d(256, 256, 3)
        self.in07_1 = InstanceNormalization(256)
        # relu
        self.refpad07_2 = nn.ReflectionPad2d(1)
        self.conv07_2 = nn.Conv2d(256, 256, 3)
        self.in07_2 = InstanceNormalization(256)
        # + input

        # res block 5
        self.refpad08_1 = nn.ReflectionPad2d(1)
        self.conv08_1 = nn.Conv2d(256, 256, 3)
        self.in08_1 = InstanceNormalization(256)
        # relu
        self.refpad08_2 = nn.ReflectionPad2d(1)
        self.conv08_2 = nn.Conv2d(256, 256, 3)
        self.in08_2 = InstanceNormalization(256)
        # + input

        # res block 6
        self.refpad09_1 = nn.ReflectionPad2d(1)
        self.conv09_1 = nn.Conv2d(256, 256, 3)
        self.in09_1 = InstanceNormalization(256)
        # relu
        self.refpad09_2 = nn.ReflectionPad2d(1)
        self.conv09_2 = nn.Conv2d(256, 256, 3)
        self.in09_2 = InstanceNormalization(256)
        # + input

        # res block 7
        self.refpad10_1 = nn.ReflectionPad2d(1)
        self.conv10_1 = nn.Conv2d(256, 256, 3)
        self.in10_1 = InstanceNormalization(256)
        # relu
        self.refpad10_2 = nn.ReflectionPad2d(1)
        self.conv10_2 = nn.Conv2d(256, 256, 3)
        self.in10_2 = InstanceNormalization(256)
        # + input

        # res block 8
        self.refpad11_1 = nn.ReflectionPad2d(1)
        self.conv11_1 = nn.Conv2d(256, 256, 3)
        self.in11_1 = InstanceNormalization(256)
        # relu
        self.refpad11_2 = nn.ReflectionPad2d(1)
        self.conv11_2 = nn.Conv2d(256, 256, 3)
        self.in11_2 = InstanceNormalization(256)
        # + input

        ##------------------------------------##
        self.deconv01_1 = nn.ConvTranspose2d(256, 128, 3, 2, 1, 1)
        self.deconv01_2 = nn.Conv2d(128, 128, 3, 1, 1)
        self.in12_1 = InstanceNormalization(128)
        # relu
        self.deconv02_1 = nn.ConvTranspose2d(128, 64, 3, 2, 1, 1)
        self.deconv02_2 = nn.Conv2d(64, 64, 3, 1, 1)
        self.in13_1 = InstanceNormalization(64)
        # relu
        self.refpad12_1 = nn.ReflectionPad2d(3)
        self.deconv03_1 = nn.Conv2d(64, 3, 7)
        # tanh

    def forward(self, x):
        y = F.relu(self.in01_1(self.conv01_1(self.refpad01_1(x))))
        y = F.relu(self.in02_1(self.conv02_2(self.conv02_1(y))))
        t04 = F.relu(self.in03_1(self.conv03_2(self.conv03_1(y))))

        ##
        y = F.relu(self.in04_1(self.conv04_1(self.refpad04_1(t04))))
        t05 = self.in04_2(self.conv04_2(self.refpad04_2(y))) + t04

        y = F.relu(self.in05_1(self.conv05_1(self.refpad05_1(t05))))
        t06 = self.in05_2(self.conv05_2(self.refpad05_2(y))) + t05

        y = F.relu(self.in06_1(self.conv06_1(self.refpad06_1(t06))))
        t07 = self.in06_2(self.conv06_2(self.refpad06_2(y))) + t06

        y = F.relu(self.in07_1(self.conv07_1(self.refpad07_1(t07))))
        t08 = self.in07_2(self.conv07_2(self.refpad07_2(y))) + t07

        y = F.relu(self.in08_1(self.conv08_1(self.refpad08_1(t08))))
        t09 = self.in08_2(self.conv08_2(self.refpad08_2(y))) + t08

        y = F.relu(self.in09_1(self.conv09_1(self.refpad09_1(t09))))
        t10 = self.in09_2(self.conv09_2(self.refpad09_2(y))) + t09

        y = F.relu(self.in10_1(self.conv10_1(self.refpad10_1(t10))))
        t11 = self.in10_2(self.conv10_2(self.refpad10_2(y))) + t10

        y = F.relu(self.in11_1(self.conv11_1(self.refpad11_1(t11))))
        y = self.in11_2(self.conv11_2(self.refpad11_2(y))) + t11
        ##

        y = F.relu(self.in12_1(self.deconv01_2(self.deconv01_1(y))))
        y = F.relu(self.in13_1(self.deconv02_2(self.deconv02_1(y))))
        y = torch.tanh(self.deconv03_1(self.refpad12_1(y)))

        return y


class InstanceNormalization(nn.Module):
    def __init__(self, dim, eps=1e-9):
        super(InstanceNormalization, self).__init__()
        self.scale = nn.Parameter(torch.FloatTensor(dim))
        self.shift = nn.Parameter(torch.FloatTensor(dim))
        self.eps = eps
        self._reset_parameters()

    def _reset_parameters(self):
        self.scale.data.uniform_()
        self.shift.data.zero_()

    def __call__(self, x):
        n = x.size(2) * x.size(3)
        t = x.view(x.size(0), x.size(1), n)
        mean = torch.mean(t, 2).unsqueeze(2).unsqueeze(3).expand_as(x)
        # Calculate the biased var. torch.var returns unbiased var
        var = torch.std(t, 2) ** 2
        var = var.unsqueeze(2).unsqueeze(3).expand_as(x) * (
            (n - 1) / torch.FloatTensor([n])
        )
        scale_broadcast = self.scale.unsqueeze(1).unsqueeze(1).unsqueeze(0)
        scale_broadcast = scale_broadcast.expand_as(x)
        shift_broadcast = self.shift.unsqueeze(1).unsqueeze(1).unsqueeze(0)
        shift_broadcast = shift_broadcast.expand_as(x)
        out = (x - mean) / torch.sqrt(var + self.eps)
        out = out * scale_broadcast + shift_broadcast
        return out

In [4]:
styles = ["Hosoda", "Hayao", "Shinkai", "Paprika"]

models = {}

for style in styles:
  model = Transformer()
  model.load_state_dict(torch.load(os.path.join("./pretrained_models", style + '_net_G_float.pth')))
  model.eval()
  models[style] = model

In [5]:
# style = "Hayao"
parser = argparse.ArgumentParser()
parser.add_argument('--input_dir', default = 'test_images')
parser.add_argument('--load_size', default = 756)
parser.add_argument('--model_path', default = './pretrained_model')
parser.add_argument('--style', default = 'Hayao')
parser.add_argument('--output_dir', default = 'cartoonized_images')
parser.add_argument('--gpu', type=int, default = 0)

_StoreAction(option_strings=['--gpu'], dest='gpu', nargs=None, const=None, default=0, type=<class 'int'>, choices=None, required=False, help=None, metavar=None)

In [6]:
opt, unknown = parser.parse_known_args()
opt

Namespace(input_dir='test_images', load_size=756, model_path='./pretrained_model', style='Hayao', output_dir='cartoonized_images', gpu=0)

In [10]:
for files in os.listdir(opt.input_dir):
	ext = os.path.splitext(files)[1]
	if ext not in ['.jpg', '.png']:
		continue
	
    # load image
	input_image = Image.open(os.path.join(opt.input_dir, files)).convert("RGB")
	
	# resize image, keep aspect ratio
	h = input_image.size[0]
	w = input_image.size[1]
	ratio = h *1.0 / w
	if ratio > 1:
		h = opt.load_size
		w = int(h*1.0/ratio)
	else:
		w = opt.load_size
		h = int(w * ratio)
	input_image = input_image.resize((h, w), Image.BICUBIC)
	input_image = np.asarray(input_image)
	
	# RGB -> BGR
	input_image = input_image[:, :, [2, 1, 0]]
	input_image = transforms.ToTensor()(input_image).unsqueeze(0)
	
    # preprocess, (-1, 1)
	input_image = -1 + 2 * input_image 
	
    # if opt.gpu > -1:
	# 	input_image = Variable(input_image, volatile=True).cuda()
	# else:
	# 	input_image = Variable(input_image, volatile=True).float()
	
    # forward
	output_image = models[opt.style](input_image)
	output_image = output_image[0]
	# BGR -> RGB
	output_image = output_image[[2, 1, 0], :, :]
	# deprocess, (0, 1)
	output_image = output_image.data.cpu().float() * 0.5 + 0.5
	
	# save
	vutils.save_image(output_image, os.path.join(opt.output_dir, files[:-4] + '_' + opt.style + '.jpg'))

In [28]:
import base64, io

img_path = os.path.join('test_images', "test.png")
input_image = Image.open(img_path)

extension = input_image.format

input_image.convert("RGB")

# Create in-memory file from the PIL image
in_mem_file = io.BytesIO()
input_image.save(in_mem_file, format = extension)

# Read the bytes
in_mem_file.seek(0)
image_bytes = in_mem_file.read()

base64_string = base64.b64encode(image_bytes).decode("utf-8")

In [10]:
f = open("demofile.txt", "r")
base64_string = f.read()

In [11]:
from base64 import b64decode
from PIL import Image
import imghdr, io

decoded_string = b64decode(base64_string)
extension = imghdr.what(None, h=decoded_string)
print(extension)

None


In [12]:
input_image = Image.open(io.BytesIO(decoded_string)).convert("RGB")

UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x000001FB5BBC6E30>

In [9]:
input_image.show()

In [30]:
# LOAD_SIZE = 756
import base64, io

def predict_fn(input_data, model):
    # logger.info('Generating cartoonization.')
	
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
	
    imgdata = base64.b64decode(input_data)
    input_image = Image.open(io.BytesIO(imgdata)).convert("RGB")

    # logger.info('Valid datatype')
    # resize image, keep aspect ratio
    # h = input_image.size[0]
    # w = input_image.size[1]
    # ratio = h *1.0 / w
    # if ratio > 1:
    #     h = LOAD_SIZE
    #     w = int(h*1.0/ratio)
    # else:
    #     w = LOAD_SIZE
    #     h = int(w * ratio)
    # input_image = input_image.resize((h, w), Image.BICUBIC)
    input_image = np.asarray(input_image)

    # RGB -> BGR
    input_image = input_image[:, :, [2, 1, 0]]
    input_image = transforms.ToTensor()(input_image).unsqueeze(0)
	
    # preprocess, (-1, 1)
    input_image = -1 + 2 * input_image 
    
    with torch.no_grad():
        input_image = input_image.to(device)
        cartoonized = model(input_image)

    output_image = cartoonized[0]
	# BGR -> RGB
    output_image = output_image[[2, 1, 0], :, :]
	# deprocess, (0, 1)
    output_image = output_image.data.cpu().float() * 0.5 + 0.5

    # logger.info('Complete prediction')
    return output_image

In [31]:
output = predict_fn(base64_string, models[opt.style])

In [32]:
vutils.save_image(output, f'test.{extension}')

In [50]:
def output_fn(prediction, response_content_type):
    # logger.info(f'Generating output. {prediction}')
    if response_content_type == 'application/json':
        transform = transforms.ToPILImage("RGB")
        pil_image = transform(prediction)

        # Create in-memory file from the PIL image
        in_mem_file = io.BytesIO()
        pil_image.save(in_mem_file, format = "PNG")

        # Read the bytes
        in_mem_file.seek(0)
        image_bytes = in_mem_file.read()

        # Convert the PIL image to bytes
        # image_bytes = pil_image.getvalue()

        # Encode the bytes to base64
        base64_string = base64.b64encode(image_bytes).decode('utf-8')
        # logger.info(f'Output generated {base64_string}')

        return base64_string

In [51]:
test = output_fn(output, 'application/json')

In [52]:
f = open("demofile3.txt", "w")
f.write(test)
f.close()

# Prepare the model for compilation

In [1]:
import tarfile, os
with tarfile.open('hayao_model.tar.gz', 'w:gz') as f:
    f.add("hayao_model.pth")
    f.add("code", arcname=os.path.basename("code"))
f.close()

In [11]:
os.path.basename("./code")

'code'

In [5]:
model_archive_name = "hayao_model.tar.gz"

In [6]:
!tar -cvpzf {model_archive_name} "./hayao_model.pth" ./code

a ./hayao_model.pth
a ./code
a ./code/serve.py
